# Import Libraries

In [ ]:
# NumPyの読み込み
import numpy as np
# Pandasの読み込み
import pandas as pd 
# OSモジュール:プログラムの中でOSの時間を利用したり、ディレクトリにアクセスしてファイルの作成/編集/削除などができる
import os
# OpenCV:画像を処理するのに必要な様々な機能を提供するライブラリ
import cv2
# matplotlib:NumPyのためのグラフ描画パッケージ
# pyplot:ほしいプロットを作るために暗黙的かつ自動的に図形や軸を作成するモジュール
from matplotlib import pyplot as plt
# TPUの場合、データはGoogle Cloud Storageのみアクセスできるため、使用
from kaggle_datasets import KaggleDatasets

# tensorflow:ニューラルネットの主なライブラリ
# keras:単体では使うことのできないライブラリ。tensorflowが必要
import tensorflow as tf
# Sequential:ニューラルネットワークの構造を簡略化するモデルの一つ
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as L
from tensorflow.keras.utils import to_categorical

# train_test_split:trainデータとtestデータに分けるモジュール
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn import metrics

%matplotlib inline

# TPU setup

In [ ]:
# 
# TPUの初期化
# 
# 例外処理のtry
try:
#     TPUのハードウェア情報を獲得。TPUが利用できない環境ではエラー
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     TPU利用可能化の確認
    print('Running on TPU:', tpu.master())
# 上記でエラー（例外）が出た場合の処理
except ValueError:
    tpu = None

# tpuが利用できる場合（Accelerator TPU）
# 上記でNoneでないとき
if tpu:
#   リモートクラスタに接続してTPUを初期化
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
#   データの並列処理を使用してトレーニングを分散する
#   TPUで並列処理の方法
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
# TPUなしのとき（Accelerator None）
else:
    strategy = tf.distribute.get_strategy()

# 並列処理のレベルに関する決定をAUTOで行う
AUTO = tf.data.experimental.AUTOTUNE
# TPUはGCS(Google Cloud Storage)にのみアクセスできる。
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# ミニバッチ勾配降下法を行う際、データセットを幾つかのサブセットに分ける
# サブセットの勾配の平均で重みを更新する
# そのサブセットの大きさをBatch Sizeと呼ぶ
# strategy.num_replicas_in_syncはstrategyの分割したTPUレプリカの数を表す。
# TPUが使用可能な場合、レプリカ数は8であるから8倍した数をバッチサイズとする。
# TPU使用不可の時、レプリカ数は1となり、バッチサイズは8
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMG_SIZE = 768

print('Batch size:', BATCH_SIZE)

# Get train and test data

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

print(train.head())

# GCSでimageのパス指定
train_path = train.image_id.apply(lambda x: f'{GCS_DS_PATH}/images/{x}.jpg').values
test_path = test.image_id.apply(lambda x: f'{GCS_DS_PATH}/images/{x}.jpg').values
# trainのimage_idを削除し、arrayに
train_label = train.loc[:, 'healthy':].values

# train_path, valid_path, train_label, valid_label = train_test_split(train_path, train_label,
#                                                                     test_size=0.1, stratify=train_label)

# Get class weights

In [ ]:
# weightをかけると全データ数の1/4になる
# n_samples[1821] / (n_classes[4] * np.bincount(y)[healthy:516/multi:91/rust:622/scab:592])
class_weight = compute_class_weight('balanced', np.unique(np.argmax(train_label, axis=1)), np.argmax(train_label, axis=1))
# barplotの作成
plt.bar(range(4), class_weight)

# Lets see some images

In [ ]:
# 2×2で表示
fig, ax = plt.subplots(2, 2)
# サンプル読み込み
img = cv2.imread('/kaggle/input/plant-pathology-2020-fgvc7/images/Train_0.jpg')
img1 = cv2.imread('/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1.jpg')
img2 = cv2.imread('/kaggle/input/plant-pathology-2020-fgvc7/images/Train_2.jpg')
img3 = cv2.imread('/kaggle/input/plant-pathology-2020-fgvc7/images/Train_3.jpg')
# 場所指定した書き出し
ax[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0, 1].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax[1, 0].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[1, 1].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))

# Decode images

In [ ]:
# デコードの定義
def decode_image(filename, label=None, image_size=(IMG_SIZE, IMG_SIZE)):
#     生データを読み込み
    bits = tf.io.read_file(filename)
#     画像のテンソルにデコード
    image = tf.image.decode_jpeg(bits, channels=3)
#     0-255のRGBを0-1にするnormalize
    image = tf.cast(image, tf.float32) / 255.0
#     1365×2048の画像を、768×768にする
    image = tf.image.resize(image, image_size)
    
#     条件式の意味はわかりませんが変換したimageをreturn
    if label is None:
        return image
    else:
        return image, label

# augmentの定義
def data_augment(image, label=None):
#     ランダムに水平方向に反転
    image = tf.image.random_flip_left_right(image)
#     ランダムに垂直方向に反転
    image = tf.image.random_flip_up_down(image)
    
#     imageをreturn
    if label is None:
        return image
    else:
        return image, label

In [ ]:
# 教師データをデコード
train_dataset = (
#     TFR形式でデータを読み書きする
    tf.data.TFRecordDataset
#     配列をスライスしてデータセットを構築する
    .from_tensor_slices((train_path, train_label))
#     データ変換（デコード）を並列化して行う
    .map(decode_image, num_parallel_calls=AUTO)
#     データ変換（増幅）を並列化する
    .map(data_augment, num_parallel_calls=AUTO)
#     データセットをメモリかローカスストレージにキャッシュできる
    .cache()
#     繰り返し。shuffleの前にあればEPOCHの境界を越えて要素がシャッフルされない
    .repeat()
#     シャッフルバッファのサイズは大きいほど完全にシャッフルされる。※ランダム要素
    .shuffle(1024)
#     バッチ化
    .batch(BATCH_SIZE)
#     CPUとTPUでそれぞれ並列に処理を実行
    .prefetch(AUTO)
)

# valid_dataset = (
#     tf.data.TFRecordDataset
#     .from_tensor_slices((valid_path, valid_label))
#     .map(decode_image, num_parallel_calls=AUTO)
#     .map(data_augment, num_parallel_calls=AUTO)
#     .cache()
#     .batch(BATCH_SIZE)
#     .prefetch(AUTO)
# )

# テストデータのデコード
test_dataset = (
    tf.data.TFRecordDataset
    .from_tensor_slices(test_path)
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

# Define the parameters

In [ ]:
EPOCHS = 40
LR_START = 0.0001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.0001
LR_RAMPUP_EPOCHS = 10
LR_SUSTAIN_EPOCHS = 4
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

lr = tf.keras.callbacks.LearningRateScheduler(lrfn)

y = [lrfn(x) for x in range(EPOCHS)]
plt.plot(y)

# EfficientNet B7-B1

In [ ]:
# # EfficientNetのインストール
# !pip install -q efficientnet

In [ ]:
# # EfficientNetB7の読み込み
# from efficientnet.tfkeras import EfficientNetB7

# # modelの作成はstrategy.scope下で行う
# with strategy.scope():
# #     全結合層を含めない/noisy-studentでself training/average pooling/画像サイズとRGB
#     efn7 = EfficientNetB7(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

# #     Sequentialモデルを作成、後ろに層を追加
#     model_efn7 = Sequential()
# #     EfficientNet層をModelに追加
#     model_efn7.add(efn7)
# #     こちらで全結合層を追加、4値分類、分類問題のため活性化関数はsoftmax
#     model_efn7.add(L.Dense(4, activation='softmax'))
# #     modelがどのように学習するかを決める
# #     最適化関数adam/損失関数は多クラス分類のためクロスエントロピー/評価関数は正解率（学習には影響しない）
#     model_efn7.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn7.summary())

In [ ]:
# from efficientnet.tfkeras import EfficientNetB6

# with strategy.scope():
#     efn6 = EfficientNetB6(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_efn6 = Sequential()
#     model_efn6.add(efn6)
#     model_efn6.add(L.Dense(4, activation='softmax'))
#     model_efn6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn6.summary())

In [ ]:
# from efficientnet.tfkeras import EfficientNetB5

# with strategy.scope():
#     efn5 = EfficientNetB5(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_efn5 = Sequential()
#     model_efn5.add(efn5)
#     model_efn5.add(L.Dense(4, activation='softmax'))
#     model_efn5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn5.summary())

In [ ]:
# from efficientnet.tfkeras import EfficientNetB4

# with strategy.scope():
#     efn4 = EfficientNetB4(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_efn4 = Sequential()
#     model_efn4.add(efn4)
#     model_efn4.add(L.Dense(4, activation='softmax'))
#     model_efn4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn4.summary())

In [ ]:
# from efficientnet.tfkeras import EfficientNetB3

# with strategy.scope():
#     efn3 = EfficientNetB3(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_efn3 = Sequential()
#     model_efn3.add(efn3)
#     model_efn3.add(L.Dense(4, activation='softmax'))
#     model_efn3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn3.summary())

In [ ]:
# from efficientnet.tfkeras import EfficientNetB2

# with strategy.scope():
#     efn2 = EfficientNetB2(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_efn2 = Sequential()
#     model_efn2.add(efn2)
#     model_efn2.add(L.Dense(4, activation='softmax'))
#     model_efn2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn2.summary())

In [ ]:
# from efficientnet.tfkeras import EfficientNetB1

# with strategy.scope():
#     efn1 = EfficientNetB1(include_top=False, weights='noisy-student', pooling='avg', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_efn1 = Sequential()
#     model_efn1.add(efn1)
#     model_efn1.add(L.Dense(4, activation='softmax'))
#     model_efn1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_efn1.summary())

# DenseNet 121/169/201

In [ ]:
# from tensorflow.keras.applications import DenseNet121

# with strategy.scope():
#     dnn121 = DenseNet121(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_dnn121 = Sequential()
#     model_dnn121.add(dnn121)
#     model_dnn121.add(L.GlobalAveragePooling2D())
#     model_dnn121.add(L.Dense(4, activation='softmax'))
#     model_dnn121.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_dnn121.summary())

In [ ]:
# from tensorflow.keras.applications import DenseNet169

# with strategy.scope():
#     dnn169 = DenseNet169(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_dnn169 = Sequential()
#     model_dnn169.add(dnn169)
#     model_dnn169.add(L.GlobalAveragePooling2D())
#     model_dnn169.add(L.Dense(4, activation='softmax'))
#     model_dnn169.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_dnn169.summary())

In [ ]:
# from tensorflow.keras.applications import DenseNet201

# with strategy.scope():
#     dnn201 = DenseNet201(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_dnn201 = Sequential()
#     model_dnn201.add(dnn201)
#     model_dnn201.add(L.GlobalAveragePooling2D())
#     model_dnn201.add(L.Dense(4, activation='softmax'))
#     model_dnn201.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_dnn201.summary())

# ResNet 50/101/152

In [ ]:
# from tensorflow.keras.applications import ResNet50

# with strategy.scope():
#     rsn50 = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_rsn50 = Sequential()
#     model_rsn50.add(rsn50)
#     model_rsn50.add(L.GlobalAveragePooling2D())
#     model_rsn50.add(L.Dense(4, activation='softmax'))
#     model_rsn50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_rsn50.summary())

In [ ]:
# from tensorflow.keras.applications import ResNet101

# with strategy.scope():
#     rsn101 = ResNet101(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_rsn101 = Sequential()
#     model_rsn101.add(rsn101)
#     model_rsn101.add(L.GlobalAveragePooling2D())
#     model_rsn101.add(L.Dense(4, activation='softmax'))
#     model_rsn101.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_rsn101.summary())

In [ ]:
# from tensorflow.keras.applications import ResNet152

# with strategy.scope():
#     rsn152 = ResNet152(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

#     model_rsn152 = Sequential()
#     model_rsn152.add(rsn152)
#     model_rsn152.add(L.GlobalAveragePooling2D())
#     model_rsn152.add(L.Dense(4, activation='softmax'))
#     model_rsn152.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model_rsn152.summary())

# VGG 16/19

In [ ]:
from tensorflow.keras.applications import VGG16

with strategy.scope():
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

    model_vgg16 = Sequential()
    model_vgg16.add(vgg16)
    model_vgg16.add(L.GlobalAveragePooling2D())
    model_vgg16.add(L.Dense(4, activation='softmax'))
    model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model_vgg16.summary())

In [ ]:
from tensorflow.keras.applications import VGG19

with strategy.scope():
    vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

    model_vgg19 = Sequential()
    model_vgg19.add(vgg19)
    model_vgg19.add(L.GlobalAveragePooling2D())
    model_vgg19.add(L.Dense(4, activation='softmax'))
    model_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model_vgg19.summary())

# Model

In [ ]:
# # Epoch終了後の各数値を監視して条件が揃った場合モデルを保存する
# # 重みのファイル名/監視する値/判定結果から保存を決定/モデルの重みを保存
# mc_efn7 = tf.keras.callbacks.ModelCheckpoint('weights_efn7.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# # 訓練の履歴の可視化
# history = model_efn7.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn7], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_efn6 = tf.keras.callbacks.ModelCheckpoint('weights_efn6.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_efn6.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn6], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_efn5 = tf.keras.callbacks.ModelCheckpoint('weights_efn5.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_efn5.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn5], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_efn4 = tf.keras.callbacks.ModelCheckpoint('weights_efn4.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_efn4.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn4], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_efn3 = tf.keras.callbacks.ModelCheckpoint('weights_efn3.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_efn3.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn3], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_efn2 = tf.keras.callbacks.ModelCheckpoint('weights_efn2.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_efn2.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn2], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_efn1 = tf.keras.callbacks.ModelCheckpoint('weights_efn1.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_efn1.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_efn1], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_dnn121 = tf.keras.callbacks.ModelCheckpoint('weights_dnn121.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_dnn121.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_dnn121], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_dnn169 = tf.keras.callbacks.ModelCheckpoint('weights_dnn169.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_dnn169.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_dnn169], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_dnn201 = tf.keras.callbacks.ModelCheckpoint('weights_dnn201.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_dnn201.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_dnn201], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_rsn50 = tf.keras.callbacks.ModelCheckpoint('weights_rsn50.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_rsn50.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_rsn50], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_rsn101 = tf.keras.callbacks.ModelCheckpoint('weights_rsn101.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_rsn101.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_rsn101], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
# mc_rsn152 = tf.keras.callbacks.ModelCheckpoint('weights_rsn152.h5', monitor='loss', save_best_only=True, save_weights_only=True)
# history = model_rsn152.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_rsn152], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
mc_vgg16 = tf.keras.callbacks.ModelCheckpoint('weights_vgg16.h5', monitor='loss', save_best_only=True, save_weights_only=True)
history = model_vgg16.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_vgg16], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
mc_vgg19 = tf.keras.callbacks.ModelCheckpoint('weights_vgg19.h5', monitor='loss', save_best_only=True, save_weights_only=True)
history = model_vgg19.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, mc_vgg19], steps_per_epoch=train_label.shape[0] // BATCH_SIZE)

In [ ]:
with strategy.scope():
#     あらかじめ保存しておいたweightパラメータをロード
#     model_efn7.load_weights('weights_efn7.h5')
#     model_efn6.load_weights('weights_efn6.h5')
#     model_efn5.load_weights('weights_efn5.h5')
#     model_efn4.load_weights('weights_efn4.h5')
#     model_efn3.load_weights('weights_efn3.h5')
#     model_efn2.load_weights('weights_efn2.h5')
#     model_efn1.load_weights('weights_efn1.h5')
#     model_dnn121.load_weights('weights_dnn121.h5')
#     model_dnn169.load_weights('weights_dnn169.h5')
#     model_dnn201.load_weights('weights_dnn201.h5')
#     model_rsn50.load_weights('weights_rsn50.h5')
#     model_rsn101.load_weights('weights_rsn101.h5')
#     model_rsn152.load_weights('weights_rsn152.h5')
    model_vgg16.load_weights('weights_vgg16.h5')
    model_vgg19.load_weights('weights_vgg19.h5')
# valid_prob = model.predict(valid_dataset, verbose=1)
# print(metrics.classification_report(np.argmax(valid_label, axis=1), np.argmax(valid_prob, axis=1)))
# print(metrics.confusion_matrix(np.argmax(valid_label, axis=1), np.argmax(valid_prob, axis=1)))

# Predict

In [ ]:
# # testデータでの予測/logの出力verbose
# probs_efn7 = model_efn7.predict(test_dataset, verbose=1)
# # probsの値をsumple_submissionの列healthy,multiple_diseases,rust,scabにあてはめ
# sub_efn7 = sub
# sub_efn7.loc[:, 'healthy':] = probs_efn7
# # CSVファイルに書き出し
# sub_efn7.to_csv('submission_efn7.csv', index=False)
# # 表示
# sub_efn7.head()

In [ ]:
# probs_efn6 = model_efn6.predict(test_dataset, verbose=1)
# sub_efn6 = sub
# sub_efn6.loc[:, 'healthy':] = probs_efn6
# sub_efn6.to_csv('submission_efn6.csv', index=False)
# sub_efn6.head()

In [ ]:
# probs_efn5 = model_efn5.predict(test_dataset, verbose=1)
# sub_efn5 = sub
# sub_efn5.loc[:, 'healthy':] = probs_efn5
# sub_efn5.to_csv('submission_efn5.csv', index=False)
# sub_efn5.head()

In [ ]:
# probs_efn4 = model_efn4.predict(test_dataset, verbose=1)
# sub_efn4 = sub
# sub_efn4.loc[:, 'healthy':] = probs_efn4
# sub_efn4.to_csv('submission_efn4.csv', index=False)
# sub_efn4.head()

In [ ]:
# probs_efn3 = model_efn3.predict(test_dataset, verbose=1)
# sub_efn3 = sub
# sub_efn3.loc[:, 'healthy':] = probs_efn3
# sub_efn3.to_csv('submission_efn3.csv', index=False)
# sub_efn3.head()

In [ ]:
# probs_efn2 = model_efn2.predict(test_dataset, verbose=1)
# sub_efn2 = sub
# sub_efn2.loc[:, 'healthy':] = probs_efn2
# sub_efn2.to_csv('submission_efn2.csv', index=False)
# sub_efn2.head()

In [ ]:
# probs_efn1 = model_efn1.predict(test_dataset, verbose=1)
# sub_efn1 = sub
# sub_efn1.loc[:, 'healthy':] = probs_efn1
# sub_efn1.to_csv('submission_efn1.csv', index=False)
# sub_efn1.head()

In [ ]:
# probs_dnn121 = model_dnn121.predict(test_dataset, verbose=1)
# sub_dnn121 = sub
# sub_dnn121.loc[:, 'healthy':] = probs_dnn121
# sub_dnn121.to_csv('submission_dnn121.csv', index=False)
# sub_dnn121.head()

In [ ]:
# probs_dnn169 = model_dnn169.predict(test_dataset, verbose=1)
# sub_dnn169 = sub
# sub_dnn169.loc[:, 'healthy':] = probs_dnn169
# sub_dnn169.to_csv('submission_dnn169.csv', index=False)
# sub_dnn169.head()

In [ ]:
# probs_dnn201 = model_dnn201.predict(test_dataset, verbose=1)
# sub_dnn201 = sub
# sub_dnn201.loc[:, 'healthy':] = probs_dnn201
# sub_dnn201.to_csv('submission_dnn201.csv', index=False)
# sub_dnn201.head()

In [ ]:
# probs_rsn50 = model_rsn50.predict(test_dataset, verbose=1)
# sub_rsn50 = sub
# sub_rsn50.loc[:, 'healthy':] = probs_rsn50
# sub_rsn50.to_csv('submission_rsn50.csv', index=False)
# sub_rsn50.head()

In [ ]:
# probs_rsn101 = model_rsn101.predict(test_dataset, verbose=1)
# sub_rsn101 = sub
# sub_rsn101.loc[:, 'healthy':] = probs_rsn101
# sub_rsn101.to_csv('submission_rsn101.csv', index=False)
# sub_rsn101.head()

In [ ]:
# probs_rsn152 = model_rsn152.predict(test_dataset, verbose=1)
# sub_rsn152 = sub
# sub_rsn152.loc[:, 'healthy':] = probs_rsn152
# sub_rsn152.to_csv('submission_rsn152.csv', index=False)
# sub_rsn152.head()

In [ ]:
probs_vgg16 = model_vgg16.predict(test_dataset, verbose=1)
sub_vgg16 = sub
sub_vgg16.loc[:, 'healthy':] = probs_vgg16
sub_vgg16.to_csv('submission_vgg16.csv', index=False)
sub_vgg16.head()

In [ ]:
probs_vgg19 = model_vgg19.predict(test_dataset, verbose=1)
sub_vgg19 = sub
sub_vgg19.loc[:, 'healthy':] = probs_vgg19
sub_vgg19.to_csv('submission_vgg19.csv', index=False)
sub_vgg19.head()